# Optimized LaTeX to PDF Converter

This notebook provides a Python script to convert LaTeX code into a PDF file using `pdflatex`. It includes optimizations to reduce processing time and handle timeouts for complex documents. The script takes LaTeX code as input, processes it, and generates a PDF output.

In [8]:
import subprocess
import os
import tempfile
import time

def latex_to_pdf(latex_code, output_filename="output.pdf", timeout_seconds=30):
    """|
    Convert LaTeX code to a PDF file with optimizations and timeout handling.
    
    Args:
        latex_code (str): The LaTeX code to convert
        output_filename (str): Name of the output PDF file
        timeout_seconds (int): Maximum time to wait for pdflatex to complete
    Returns:
        bool: True if successful, False if an error occurs
    """
    start_time = time.time()
    try:
        # Create a temporary directory to store intermediate files
        with tempfile.TemporaryDirectory() as temp_dir:
            # Define paths for temporary files
            tex_path = os.path.join(temp_dir, "temp.tex")
            pdf_path = os.path.join(temp_dir, "temp.pdf")
            
            # Write LaTeX code to a temporary .tex file
            with open(tex_path, "w", encoding="utf-8") as tex_file:
                tex_file.write(latex_code)
            
            # Run pdflatex with non-interactive mode to avoid hangs
            result = subprocess.run(
                ["pdflatex", "-interaction=nonstopmode", "-output-directory", temp_dir, tex_path],
                capture_output=True,
                text=True,
                check=True,
                timeout=timeout_seconds
            )
            
            # Check if PDF was created
            if not os.path.exists(pdf_path):
                print("Error: PDF file was not created.")
                print(result.stderr)
                return False
            
            # Move the PDF to the desired output location
            os.rename(pdf_path, output_filename)
            elapsed_time = time.time() - start_time
            print(f"PDF successfully created: {output_filename} in {elapsed_time:.2f} seconds")
            return True
            
    except subprocess.TimeoutExpired:
        print(f"Error: pdflatex took too long and was terminated after {timeout_seconds} seconds.")
        return False
    except subprocess.CalledProcessError as e:
        print(f"Error running pdflatex: {e.stderr}")
        return False
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return False

## Example Usage

Below is an example of how to use the `latex_to_pdf` function with a sample LaTeX code.

In [11]:
# Sample LaTeX code
sample_latex = r"""
\documentclass{article}
\usepackage[utf8]{inputenc}
\begin{document}
\title{Sample LaTeX Document}
\author{Author Name}
\maketitle
\section{Introduction}
This is a sample LaTeX document converted to PDF using Python.
\end{document}
"""

# Convert to PDF
latex_to_pdf(sample_latex, "sample3.pdf", timeout_seconds=30)

PDF successfully created: sample3.pdf in 0.46 seconds


True

## Interactive LaTeX Input

You can provide your own LaTeX code by modifying the `custom_latex` variable below. For large documents, consider reading from a file to avoid slow input handling.

In [ ]:
# Define your custom LaTeX code here
custom_latex = r"""
\documentclass{article}
\usepackage[utf8]{inputenc}
\begin{document}
Hello, this is my custom LaTeX document!
\end{document}
"""

# Convert custom LaTeX to PDF
latex_to_pdf(custom_latex, "custom_output.pdf", timeout_seconds=30)

## Reading LaTeX from a File

For large LaTeX documents, reading from a file can be faster than manual input. Save your LaTeX code in a `.tex` file and use the cell below.

In [ ]:
# Read LaTeX code from a file
input_file = "path/to/your/document.tex"  # Replace with your .tex file path
try:
    with open(input_file, "r", encoding="utf-8") as tex_file:
        latex_code = tex_file.read()
    latex_to_pdf(latex_code, "file_output.pdf", timeout_seconds=30)
except FileNotFoundError:
    print(f"Error: File {input_file} not found.")

## Optimizations and Troubleshooting

### Why is it slow?
- **Complex LaTeX Code**: Large documents, heavy use of packages (e.g., `tikz`, `pgfplots`), or many images can slow down `pdflatex`.
- **System Resources**: Limited CPU or memory can bottleneck the process.
- **Package Dependencies**: Missing or outdated LaTeX packages can cause errors or slow compilation.
- **I/O Operations**: Writing and reading large files or images can add overhead.

### Optimizations Included
- **Non-Interactive Mode**: Uses `-interaction=nonstopmode` to prevent `pdflatex` from hanging on errors.
- **Timeout Handling**: Terminates `pdflatex` if it exceeds the specified time (default: 30 seconds).
- **Temporary Directory**: Uses `tempfile` to manage intermediate files efficiently and clean up automatically.
- **Elapsed Time Reporting**: Measures and displays the time taken for conversion.

### Tips to Speed Up
1. **Simplify LaTeX Code**:
   - Avoid unnecessary packages or complex graphics.
   - Use `\includegraphics` with compressed images (e.g., PNG or JPEG instead of high-resolution PDFs).
   - Minimize `tikz` or `pgfplots` usage or pre-render complex figures.
2. **Use a Faster LaTeX Engine**:
   - Replace `pdflatex` with `lualatex` or `xelatex` for certain documents (modify the `subprocess.run` command accordingly).
   - Example: `["lualatex", "-interaction=nonstopmode", "-output-directory", temp_dir, tex_path]`
3. **Precompile Packages**:
   - Use `mylatexformat` to precompile frequently used packages.
4. **Optimize Images**:
   - Compress images before including them in LaTeX.
   - Use tools like `ImageMagick` to reduce image size.
5. **Check System Resources**:
   - Ensure your system has sufficient CPU and memory.
   - Close unnecessary applications during compilation.
6. **Update LaTeX Distribution**:
   - Ensure you have the latest version of TeX Live or MiKTeX for performance improvements.
7. **Debug Slow Compilation**:
   - Check the `pdflatex` output (stored in `result.stderr`) for warnings about slow operations.
   - Look for missing fonts or packages causing delays.

### Notes
- Ensure `pdflatex` is installed and accessible from the command line (e.g., part of TeX Live or MiKTeX).
- The output PDF will be saved in the same directory as this notebook.
- For very large documents, consider breaking them into smaller parts or using `include`/`input` to compile sections separately.
- If timeouts persist, increase `timeout_seconds` or simplify the LaTeX code.